### Ideas worked upon: 


1. Length of story : Longer the story, harder to understand
2. Length of sentence : Longer the sentence, harder to understand
3. Length Of Words : Longer the Words, harder to understand
4. Frequency of words : Lesser the frequency, more the unfamiliarity of the words

Based on these data points we can work on the different indexes for readability like Gunning fog index or Flesch Kincaid indexes or create our own indexes based in ranges to decide what can be the difficulty for the story.

In [245]:
import pandas as pd
import numpy as np
pd.options.display.max_colwidth = 10000

In [246]:
import glob   
from statistics import mean

In [247]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

In [248]:
import re

In [249]:
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer() 

In [250]:
stop_words = set(stopwords.words('english')) 
len(stop_words)

179

#### Location to the path where files are stored

In [251]:
path = './Story text files/*.txt'   
files=glob.glob(path) 

In [252]:
files[359]

'./Story text files/541.txt'

#### Reading the files

In [253]:
data = []
for fileToRead in files:
    with open(fileToRead, 'r') as file:
        data.append(file.read())

In [254]:
data[105]

''

In [255]:
dataDf = pd.DataFrame(data)

In [256]:
dataDf['fileName'] = files

In [257]:
dataDf.columns = ['Text', 'fileName']

In [258]:
regexp = re.compile('[0-9]|-')

In [259]:
dataDf.loc[:,'Text'] = dataDf.Text.apply(lambda x: regexp.sub('',x))

In [260]:
dataDf.loc[:, 'StoryLength'] = dataDf.Text.apply(lambda x: len(x))

##### Some files were empty, thus removing those files

In [261]:
dataDf.drop(dataDf[dataDf.StoryLength == 0].index, axis = 0, inplace = True)

In [262]:
min(dataDf.StoryLength)

13

In [263]:
dataDf[dataDf.StoryLength==13]

Text                    fileName  StoryLength
359  BOW!MEOW!WOW!  ./Story text files/541.txt           13

In [264]:
dataDf.Text.head().apply(lambda x: [j for j in x.split() if j.lower() not in stop_words])

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  [Four, friends, want, race, toys.Veen, green, car., Meena, yellow, auto.Sanju, red, bus., Manju, blue, lorry.Ready, ,, GO!, comes, Lucky., Haha,, Lucky, joined, race.Look,, look!, Lucky, fastest, all., Lucky, wins, race.]
1                 [Rabbit, sleeping, apple, tree., apple, fell, branch.A, voice, said,, "Run, Rabbit, run!, ", woke, quickly, ran, away, great, speed.She, met, Chicken., "Why, running?, ", asked, Chicken., Rabbit, replied,, "I, kno

In [265]:
dataDf.reset_index(drop=True, inplace = True)

#### Word Length Features

In [266]:
dataDf['wordLength'] = dataDf.Text.apply(lambda x: [len(j) for j in x.split() if j.lower() not in stop_words])

In [267]:
# Getting the longest length word in the story
dataDf['maxWordLength'] = dataDf.wordLength.apply(lambda x: max(x))

In [268]:
# Getting the average word length in the story
dataDf['avgWordLength'] = dataDf.wordLength.apply(lambda x: mean(x))

In [269]:
# Taking words with length smaller than 8 i.e. we are segregating smaller and longer words as longer 
# words might be difficult to read
dataDf['smallWords'] = dataDf.wordLength.apply(lambda x: sum(np.array(x) < 8))  
dataDf['largeWords'] = dataDf.wordLength.apply(lambda x: sum(np.array(x) >= 8))   

#### Sentence Length Character Features

In [270]:
dataDf['sentenceLengthChar'] = dataDf.Text.apply(lambda x: [len(j) for j in re.split("!|\.|\?", x)])

In [271]:
dataDf['maxSentenceLengthChar'] = dataDf.sentenceLengthChar.apply(lambda x: max(x))

In [272]:
dataDf['avgSentenceLengthChar'] = dataDf.sentenceLengthChar.apply(lambda x: mean(x))

In [273]:
# Taking sentences with length smaller than 50 chars i.e. we are segregating smaller and longer sentences as longer 
# sentences might be difficult to read
dataDf['smallSentenceChar'] = dataDf.sentenceLengthChar.apply(lambda x: sum(np.array(x) < 50))  
dataDf['largeSentenceChar'] = dataDf.sentenceLengthChar.apply(lambda x: sum(np.array(x) >= 50))   

#### Sentence Length Word Features

In [274]:
dataDf['sentenceLengthWord'] = dataDf.Text.apply(lambda x: [len(re.findall(" ", j)) + 1 for j in re.split("!|\.|\?", x)])

In [275]:
dataDf['maxSentenceLengthWord'] = dataDf.sentenceLengthWord.apply(lambda x: max(x))

In [276]:
dataDf['avgSentenceLengthWord'] = dataDf.sentenceLengthWord.apply(lambda x: mean(x))

In [277]:
# Taking sentences with length smaller than 9 words i.e. we are segregating smaller and longer sentences as longer 
# sentences might be difficult to read
dataDf['smallSentenceWord'] = dataDf.sentenceLengthWord.apply(lambda x: sum(np.array(x) < 9))  
dataDf['largeSentenceWord'] = dataDf.sentenceLengthWord.apply(lambda x: sum(np.array(x) >= 9))   

#### Finding rare words in the text documets can help us in deciding which words might be less familiar or occur less number of times

In [278]:
wordFreq = {}
for row in dataDf.Text.iteritems():
    for word in set(row[1].split()):
        lemmatizedWord = lemmatizer.lemmatize(word)
        if lemmatizedWord.lower():
            if lemmatizedWord not in wordFreq:
                wordFreq[lemmatizedWord] = 1
            else:
                wordFreq[lemmatizedWord] += 1

In [279]:
lemmatizer.lemmatize('being').lower() in stop_words

True

In [280]:
wordFreq['being']

89

In [281]:
sorted(wordFreq.items(), key = lambda item: item[1], reverse=True) 

[('a', 801),
 ('the', 531),
 ('and', 510),
 ('to', 508),
 ('it', 472),
 ('is', 463),
 ('in', 461),
 ('of', 438),
 ('you', 418),
 ('with', 407),
 ('are', 393),
 ('on', 388),
 ('for', 376),
 ('I', 375),
 ('like', 368),
 ('The', 363),
 ('that', 361),
 ('all', 356),
 ('at', 347),
 ('not', 342),
 ('up', 338),
 ('wa', 327),
 ('from', 324),
 ('have', 322),
 ('her', 322),
 ('one', 316),
 ('they', 310),
 ('can', 304),
 ('make', 300),
 ('she', 296),
 ('go', 296),
 ('this', 290),
 ('out', 288),
 ('be', 283),
 ('will', 277),
 ('get', 276),
 ('so', 275),
 ('into', 270),
 ('he', 269),
 ('It', 267),
 ('his', 266),
 ('when', 263),
 ('She', 262),
 ('very', 262),
 ('But', 262),
 ('do', 261),
 ('ha', 260),
 ('had', 259),
 ('see', 256),
 ('their', 252),
 ('come', 250),
 ('my', 250),
 ('They', 246),
 ('He', 245),
 ('And', 244),
 ('.', 243),
 ('what', 233),
 ('them', 232),
 ('but', 232),
 ('your', 232),
 ('just', 230),
 ('by', 227),
 ('look', 225),
 ('were', 222),
 ('say', 221),
 ('an', 219),
 ('who', 218),

In [282]:
text = "I am being shy"
[wordFreq[lemmatizer.lemmatize(j)] for j in text.split()]

[375, 143, 89, 5]

In [283]:
dataDf['wordFreqCorpus'] = dataDf.Text.apply(lambda x: [wordFreq[lemmatizer.lemmatize(j)] for j in x.split() if j.lower() not in stop_words])

In [284]:
# Assuming words occuring less than 10 number of times as the less familiar words

dataDf['familiarWords'] = dataDf.wordFreqCorpus.apply(lambda x: sum(np.array(x) >= 10))
dataDf['nonFamiliarWords'] = dataDf.wordFreqCorpus.apply(lambda x: sum(np.array(x) < 10))



#### Syllable Count

In [285]:
# Syllable count helps us in deciding if the word is difficult to understand

def syllable_count(word):
    if(len(word) == 0):
        return 0
    count = 0
    vowels = "aeiouyAEIOUY"
    if word[0] in vowels:
        count += 1
    for index in range(1, len(word)):
        if word[index] in vowels and word[index - 1] not in vowels:
            count += 1
    if word.endswith("e"):
        count -= 1
    if count == 0:
        count += 1
    return count



In [288]:
dataDf['syllableCount'] = dataDf.Text.apply(lambda x: [syllable_count(j) for j in re.split("!|\.|\?|\ ", x) if j.lower() not in stop_words])

In [289]:
dataDf['maxSyllableCount'] = dataDf.syllableCount.apply(lambda x: max(x))

In [290]:
dataDf['avgSyllableCount'] = dataDf.syllableCount.apply(lambda x: mean(x))

In [291]:
# Number of syllables will also help in deciding complexity of the text
dataDf['smallSyllableCount'] = dataDf.syllableCount.apply(lambda x: sum(np.array(x) < 3))  
dataDf['largeSyllableCount'] = dataDf.syllableCount.apply(lambda x: sum(np.array(x) >= 3))   

In [292]:
dataDf.columns

Index(['Text', 'fileName', 'StoryLength', 'wordLength', 'maxWordLength',
       'avgWordLength', 'smallWords', 'largeWords', 'sentenceLengthChar',
       'maxSentenceLengthChar', 'avgSentenceLengthChar', 'smallSentenceChar',
       'largeSentenceChar', 'sentenceLengthWord', 'maxSentenceLengthWord',
       'avgSentenceLengthWord', 'smallSentenceWord', 'largeSentenceWord',
       'wordFreqCorpus', 'familiarWords', 'nonFamiliarWords', 'syllableCount',
       'maxSyllableCount', 'avgSyllableCount', 'smallSyllableCount',
       'largeSyllableCount'],
      dtype='object')

In [293]:
dataDf.reset_index(inplace=True, drop=True)

### Saving to a file

In [295]:
dataDf.to_csv?

In [297]:
dataDf[['Text', 'fileName', 'StoryLength', 'maxWordLength', 'avgWordLength','smallWords', 'largeWords',
       'maxSentenceLengthChar', 'avgSentenceLengthChar', 'smallSentenceChar',
       'largeSentenceChar', 'maxSentenceLengthWord','avgSentenceLengthWord', 
       'smallSentenceWord', 'largeSentenceWord','familiarWords', 'nonFamiliarWords',
       'maxSyllableCount', 'avgSyllableCount', 'smallSyllableCount',
       'largeSyllableCount']].to_csv('processedOutput.tsv', sep = '\t',index = False, line_terminator='\n')